<a href="https://www.kaggle.com/code/guywaffo/notebookfd0440b2c3?scriptVersionId=103300109" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Network

In [1]:
# Inspiration from AlexNet CNNF
'''The network consists of 5 Convolutional (CONV) layers and 3 Fully Connected (FC) layers.
The activation used is the Rectified Linear Unit (ReLU).
params:
data_in-shape : Tensor containing the ppg time series. Size (batch_size, ppg_length, 1) batch_size=128
fs : sampling frequency (default fs = 125 Hz)
'''
#Import libraries
import tensorflow as tf
from keras.layers import Reshape
from tensorflow.keras.layers import Softmax,Bidirectional,LSTM, Permute, Input, Add, Conv1D, MaxPooling1D, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling1D, MaxPooling2D, GlobalMaxPooling2D, LeakyReLU, GlobalAveragePooling2D, ReLU, Dropout
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model


def AlexNet_1D(data_in_shape, num_output=2, dil=1, kernel_size=3, fs = 125, useMaxPooling=True, UseDerivative=False):

    # Define the input as a tensor with shape input_shape
    X_input = Input(shape=data_in_shape)

    if UseDerivative:
        dt1 = (X_input[:,1:] - X_input[:,:-1])*fs
        dt2 = (dt1[:,1:] - dt1[:,:-1])*fs

        dt1 = tf.pad(dt1, tf.constant([[0,0],[0,1],[0,0]]))
        dt2 = tf.pad(dt2, tf.constant([[0,0],[0,2],[0,0]]))
        X = tf.concat([X_input, dt1, dt2], axis=2)
    else:        # X=tf.keras.layers.Concatenate(axis=2)([X_input, dt1,dt2])
        X=X_input

    # convolutional stage
    X = Conv1D(filters=2, kernel_size=350, strides=1, name='conv1', kernel_initializer=glorot_uniform(seed=0), padding="same")(X)
    X = Activation(ReLU())(X)
    X = BatchNormalization(axis=-1, name='BatchNorm1')(X)
    X = MaxPooling1D(175, strides=1, name="MaxPool1",padding="same")(X)


    X = Conv1D(filters=10, kernel_size=175, strides=1, name='conv2', kernel_initializer=glorot_uniform(seed=0), padding="same")(X)
    X = Activation(ReLU())(X)
    X = BatchNormalization(axis=-1, name='BatchNorm2')(X)
    X = MaxPooling1D(25, strides=1, name="MaxPool2",padding="same")(X)

    X = Conv1D(filters=20, kernel_size=25, strides=1, name='conv3', kernel_initializer=glorot_uniform(seed=0), padding="same")(X)
    X = Activation(ReLU())(X)
    X = BatchNormalization(axis=-1, name='BatchNorm3')(X)
    X = MaxPooling1D(10, strides=1, name="Maxpool3",padding="same")(X)

    X = Conv1D(filters=40, kernel_size=10, strides=1, name='conv4', kernel_initializer=glorot_uniform(seed=0), padding="same")(X)
    X = Activation(ReLU())(X)
    X = BatchNormalization(axis=-1, name='BatchNorm4')(X)
    X = MaxPooling1D(4, strides=1, name="Maxpool4",padding="same")(X)

    #Flattening the output of the CNN
    X= Flatten()(X)
    X= Reshape((875,40))(X)
    #Bi-LSTM stage
    X = Bidirectional(LSTM(128, return_sequences=True),merge_mode='concat')(X)
    X = Bidirectional(LSTM(350, return_sequences=True),merge_mode='concat')(X)

    # Fully connected slayer
    X = Flatten()(X)
    X = Dense( 2, activation='relu', name='dense', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Dropout(rate=0.5)(X)


    # output stage
    X_SBP = Dense(1, activation='relu', name='SBP', kernel_initializer=glorot_uniform(seed=0))(X)
    X_DBP = Dense(1, activation='relu', name='DBP', kernel_initializer=glorot_uniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=[X_SBP, X_DBP], name='AlexNet_1D')
    return model



## Training


In [2]:
""" train neural architectures using PPG data

This script trains a neural network using PPG data. The data is loaded from the the .tfrecord files created by the script
'hdf_to_tfrecord.py'.
"""
import csv
import os.path
import warnings
from os.path import expanduser, join
from os import environ
from sys import argv
from functools import partial
from datetime import datetime
import argparse

import tensorflow as tf
# tf.compat.v1.disable_eager_execution()


import pandas as pd
import numpy as np
import glob

from tqdm import tqdm



gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

BATCH_SIZE=54
WIN_LEN=875
def read_tfrecord(example, win_len=WIN_LEN):
    tfrecord_format = (
        {
            'ppg': tf.io.FixedLenFeature([win_len], tf.float32),
            'label': tf.io.FixedLenFeature([2], tf.float32)
        }
    )
    parsed_features = tf.io.parse_single_example(example, tfrecord_format)

    return parsed_features['ppg'], (parsed_features['label'][0], parsed_features['label'][1])


def create_dataset(tfrecords_dir, tfrecord_basename, win_len=WIN_LEN, batch_size=BATCH_SIZE, modus='train'):
    # pattern = join(tfrecords_dir, modus, tfrecord_basename + "_" + modus + "_?????_of_?????.tfrecord")
    pattern = glob.glob(tfrecords_dir + f"/{modus}/*.tfrecord")
    dataset = tf.data.TFRecordDataset.from_tensor_slices(pattern)

    if modus == 'train':
        dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
        dataset = dataset.interleave(
            tf.data.TFRecordDataset,
            cycle_length=800,
            block_length=400)
    else:
        dataset = dataset.interleave(
            tf.data.TFRecordDataset)

    dataset = dataset.map(partial(read_tfrecord, win_len=win_len), num_parallel_calls=2)
    dataset = dataset.shuffle(4096, reshuffle_each_iteration=True)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.repeat()

    return dataset


def get_model(architecture, input_shape, UseDerivative=False):
    return {
        'alexnet': AlexNet_1D(input_shape, UseDerivative=UseDerivative),
    }[architecture]


def ppg_train_mimic_iii(architecture,
                        DataDir,
                        ResultsDir,
                        CheckpointDir,
                        tensorboard_tag,
                        tfrecord_basename,
                        experiment_name,
                        win_len=WIN_LEN,
                        batch_size=BATCH_SIZE,
                        lr=None,
                        N_epochs=20,
                        Ntrain=1e6,
                        Nval=2.5e5,
                        Ntest=2.5e5,
                        UseDerivative=False,
                        earlystopping=True):
    # create datasets for training, validation and testing using .tfrecord files
    test_dataset = create_dataset(DataDir, tfrecord_basename, win_len=win_len, batch_size=batch_size,
                                  modus='test')
    train_dataset = create_dataset(DataDir, tfrecord_basename, win_len=win_len, batch_size=batch_size, modus='train')
    val_dataset = create_dataset(DataDir, tfrecord_basename, win_len=win_len, batch_size=batch_size,
                                 modus='val')
    data_in_shape = (win_len, 1)

    # load the neurarchitecture
    model = get_model(architecture, data_in_shape, UseDerivative=UseDerivative)

    # callback for logging training and validation results
    csvLogger_cb = tf.keras.callbacks.CSVLogger(
        filename=join(ResultsDir, experiment_name + '_learningcurve.csv')
    )

    # checkpoint callback
    cb_file=join(CheckpointDir, experiment_name + '_cb.h5')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=cb_file,
        save_best_only=True
    )

    if os.path.exists(cb_file):
        model.load_weights(cb_file)
        print("Loaded weights from checkpoint")

    # tensorboard callback
    tensorbard_cb = tf.keras.callbacks.TensorBoard(
        log_dir=join(ResultsDir, 'tb', tensorboard_tag),
        histogram_freq=0,
        update_freq="batch",
        write_images=True,
        write_graph=True    )

    # callback for early stopping if validation loss stops improving
    EarlyStopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )


    # define Adam optimizer
    if lr is None:
        opt = tf.keras.optimizers.Adam()
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=lr)

    # compile model using mean squared error as loss function
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.mean_squared_error,
        metrics=[['mae'], ['mae']]
    )

    class LRA(tf.keras.callbacks.Callback):
        def __init__(self, model, initial_learning_rate,frequency=1000,gamma=0.1):
            super(LRA, self).__init__()
            self.current_learning_rate=initial_learning_rate
            self.model=model
            self.frequency=frequency
            self.current_iter=0
            self.gamma=gamma

        def on_train_begin(self, logs=None):
            tf.keras.backend.set_value(self.model.optimizer.lr,
                                       self.current_learning_rate)

        def on_train_batch_end(self, batch, logs=None):
            self.current_iter+=1
            if self.current_iter%self.frequency==0:
                self.current_learning_rate=self.current_learning_rate-self.current_learning_rate*self.gamma
                tf.keras.backend.set_value(self.model.optimizer.lr, self.current_learning_rate)
                
                # print("Updating the learning rate to: ",self.current_learning_rate)


    cb_list = [checkpoint_cb,
               tensorbard_cb,
               csvLogger_cb,
               EarlyStopping_cb if earlystopping == True else [],
               LRA(model,lr,gamma=0.1)]





    # # Perform Training and Validation
    history = model.fit(
        train_dataset,
        steps_per_epoch=Ntrain // batch_size,
        epochs=N_EPOCHS,
        validation_data=val_dataset,
        validation_steps=Nval // batch_size,
        callbacks=cb_list
    )
    # Predictions on the testset
    print("Loading weights from ", checkpoint_cb.filepath)
    model.load_weights(checkpoint_cb.filepath)
    test_results = pd.DataFrame({'SBP_true': [],
                                 'DBP_true': [],
                                 'SBP_est': [],
                                 'DBP_est': []})

    # store predictions on the test set as well as the corresponding ground truth in a csv file
    test_dataset = iter(test_dataset)
    for i in tqdm(range(int(Ntest // batch_size)), "Running test"):
        ppg_test, BP_true = test_dataset.next()
        BP_est = model.predict(ppg_test, verbose=0)
        with warnings.catch_warnings():
            TestBatchResult = pd.DataFrame({'SBP_true': BP_true[0].numpy(),
                                            'DBP_true': BP_true[1].numpy(),
                                            'SBP_est': np.squeeze(BP_est[0]),
                                            'DBP_est': np.squeeze(BP_est[1]),
                                            })
            test_results = test_results.append(TestBatchResult)

    ResultsFile = join(ResultsDir, experiment_name + '_test_results.csv')
    test_results.to_csv(ResultsFile)

    ResultsFileMae = join(ResultsDir, experiment_name + '_test_results_ae.csv')

    sbp_mae = np.mean(np.abs(test_results["SBP_true"] - test_results["SBP_est"]))
    sbp_aestd = np.std(np.abs(test_results["SBP_true"] - test_results["SBP_est"]))

    dbp_mae = np.mean(np.abs(test_results["DBP_true"] - test_results["DBP_est"]))
    dbp_aestd = np.std(np.abs(test_results["DBP_true"] - test_results["DBP_est"]))

    with open(ResultsFileMae, "w") as output:
        writer = csv.writer(output)
        writer.writerow(["sbp_mae", "sbp_aestd", "dbp_mae", "dbp_aestd"])
        writer.writerow([sbp_mae, sbp_aestd, dbp_mae, dbp_aestd])
    test_results.to_csv(ResultsFile)

    idx_min = np.argmin(history.history['val_loss'])

    print(' Training finished')

    return history.history['SBP_mae'][idx_min], history.history['DBP_mae'][idx_min], history.history['val_SBP_mae'][
        idx_min], history.history['val_DBP_mae'][idx_min]


2022-08-14 21:10:52.364015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:10:52.484612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:10:52.485424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Launch training

In [3]:
architecture = "alexnet"
experiment_name = "training_kaggle"
experiment_name = datetime.now().strftime("%Y-%d-%m") + '_' + architecture + '_' + experiment_name
# experiment_name="2022-08-08_alexnet_training4"
ROOT_DIR="/kaggle/working/logs"
DataDir = "/kaggle/input/mmicdataset/train_test"
ResultsDir = os.path.join(ROOT_DIR, "results")
CheckpointDir = os.path.join(ROOT_DIR,"checkpoints")
tb_tag = experiment_name
lr = 0.001
batch_size = 256
WIN_LEN = 875
N_EPOCHS = 20


tfrecord_basename = 'MIMIC_III_ppg'

ppg_train_mimic_iii(architecture,
                    DataDir,
                    ResultsDir,
                    CheckpointDir,
                    tb_tag,
                    tfrecord_basename,
                    experiment_name,
                    win_len=WIN_LEN,
                    batch_size=batch_size,
                    lr=lr,
                    N_epochs=N_EPOCHS,
                    UseDerivative=False,
                    earlystopping=True)  # False originally


2022-08-14 21:10:52.586403: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-14 21:10:52.586746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:10:52.587480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:10:52.588137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Epoch 1/20


2022-08-14 21:11:01.889054: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-14 21:11:04.123144: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


   1/3906 [..............................] - ETA: 15:48:36 - loss: 19264.7031 - SBP_loss: 15146.2100 - DBP_loss: 4118.4922 - SBP_mae: 120.5801 - DBP_mae: 62.6778

2022-08-14 21:11:11.744403: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-14 21:11:11.744481: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


...

2022-08-14 21:11:13.139130: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-14 21:11:13.153899: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-08-14 21:11:13.299908: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 18324 callback api events and 18318 activity events. 
2022-08-14 21:11:13.519823: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-14 21:11:13.932742: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /kaggle/working/logs/results/tb/2022-14-08_alexnet_training_kaggle/train/plugins/profile/2022_08_14_21_11_13

2022-08-14 21:11:14.194694: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /kaggle/working/logs/results/tb/2022-14-08_alexnet_training_kaggle/train/plugins/profile/2022_08_14_21_11_13/73cf64a2c374.trace.js

3906/3906 [==============================] - 3207s 817ms/step - loss: 9797.4248 - SBP_loss: 7830.3560 - DBP_loss: 1967.0708 - SBP_mae: 69.7661 - DBP_mae: 35.2498 - val_loss: 5086.8716 - val_SBP_loss: 4058.9177 - val_DBP_loss: 1027.9537 - val_SBP_mae: 59.5346 - val_DBP_mae: 29.5245
Epoch 2/20
3906/3906 [==============================] - 3190s 817ms/step - loss: 9253.2080 - SBP_loss: 7466.6084 - DBP_loss: 1786.6019 - SBP_mae: 67.8288 - DBP_mae: 33.4075 - val_loss: 4745.9351 - val_SBP_loss: 3802.8704 - val_DBP_loss: 943.0596 - val_SBP_mae: 57.4279 - val_DBP_mae: 28.0783
Epoch 3/20
3906/3906 [==============================] - 3191s 817ms/step - loss: 8940.2373 - SBP_loss: 7254.1792 - DBP_loss: 1686.0647 - SBP_mae: 66.8498 - DBP_mae: 32.4889 - val_loss: 4409.2700 - val_SBP_loss: 3546.8953 - val_DBP_loss: 862.3798 - val_SBP_mae: 55.1772 - val_DBP_mae: 26.6093
Epoch 4/20
3906/3906 [==============================] - 3190s 817ms/step - loss: 8734.2324 - SBP_loss: 7114.0825 - DBP_loss: 1620.1575

Running test: 100%|██████████| 976/976 [18:43<00:00,  1.15s/it]


 Training finished


(66.8497543334961, 32.48891830444336, 55.177162170410156, 26.609338760375977)